#### --- Import Libraries ---

In [61]:
import pandas as pd
import cv2
import os
import random
from sklearn.model_selection import train_test_split

#### --- Config & Setup ---

In [ ]:
import os
project_root = os.getcwd()  # Gets your current working directory


In [63]:
# File paths
S2_PATH = os.path.join(project_root, 'dataset', 'annotations', 'pone.0231608.s002.xlsx')
S3_PATH = os.path.join(project_root, 'dataset', 'annotations', 'pone.0231608.s003.xlsx')
VIDEOS_DIR = os.path.join(project_root, 'dataset', 'videos')
CLIPS_DIR = os.path.join(project_root, 'outputs', 'clips')

# Create all needed subfolders (eyes/chin/negative/csv) robustly
folders = [
    os.path.join(CLIPS_DIR, 'eyes_positive'),
    os.path.join(CLIPS_DIR, 'chin_positive'),
    os.path.join(CLIPS_DIR, 'negative'),
    os.path.join(project_root, 'outputs', 'csv')
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("All folders and paths set up correctly.")
print("S2_PATH:", S2_PATH)
print("S3_PATH:", S3_PATH)
print("VIDEOS_DIR:", VIDEOS_DIR)
print("CLIPS_DIR:", CLIPS_DIR)



All folders and paths set up correctly.
S2_PATH: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\annotations\pone.0231608.s002.xlsx
S3_PATH: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\annotations\pone.0231608.s003.xlsx
VIDEOS_DIR: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\videos
CLIPS_DIR: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\outputs\clips


In [69]:
s2_raw = pd.read_excel(S2_PATH, sheet_name='FACS', header=None)
print("First 15 rows of raw S3 annotation:")
print(s2_raw.head(15))

First 15 rows of raw S3 annotation:
                          0        1   2             3   4             5   \
0                        NaN  FILM 1  NaN           NaN NaN           NaN   
1   Start/End time: hh:mm:ss     Code NaN  Duration (s) NaN   Start time    
2                        NaN    VC71R NaN         28.63 NaN  00:00:00.000   
3                        NaN    VC72R NaN         28,63 NaN  00:00:00.000   
4                        NaN     AD51 NaN          1,82 NaN  00:00:00.100   
5                        NaN    AU47L NaN          0,23 NaN  00:00:00.220   
6                        NaN    AU101 NaN          1,48 NaN  00:00:00.470   
7                        NaN     AD51 NaN          0,18 NaN  00:00:01.874   
8                        NaN    AU47L NaN          0,37 NaN  00:00:01.930   
9                        NaN     AD1L NaN          0,71 NaN  00:00:02.290   
10                       NaN  EAD104R NaN          0,76 NaN  00:00:02.300   
11                       NaN   AU101L Na

In [40]:
s3_raw = pd.read_excel(S3_PATH, sheet_name='FACS', header=None)
print("First 15 rows of raw S3 annotation:")
print(s3_raw.head(15))
# View any actual Media rows, AU codes, columns, etc.


First 15 rows of raw S3 annotation:
           0                            1                2                3  \
0   Media 1   Annotation date: 2019-03-22              NaN              NaN   
1        NaN                          NaN              NaN              NaN   
2        NaN                          NaN              NaN              NaN   
3      Tier                           NaN      Begin time         End time    
4       Ears                          NaN  00:00:00.640000  00:00:01.580000   
5       Ears                          NaN  00:00:01.730000  00:00:02.300000   
6       Ears                          NaN  00:00:04.480000  00:00:05.100000   
7       Ears                          NaN  00:00:06.170000  00:00:06.740000   
8       Ears                          NaN  00:00:06.780000  00:00:07.320000   
9       Ears                          NaN  00:00:07.700000  00:00:07.960000   
10      Ears                          NaN  00:00:07.960000  00:00:08.220000   
11      Ears    

In [76]:
# Logic: Main 1-12 as S1-S12; extras cycle (e.g., Film 17 → S5, Film 22 → S10, etc.)
base_videos = ['S1_Video.mp4', 'S2_Video.mp4', 'S3_Video.mp4', 'S4_Video.mp4', 'S5_Video.mp4', 'S6_Video.mp4', 
               'S7_Video.mp4', 'S8_Video.mp4', 'S9_Video.mp4', 'S10_Video.mp4', 'S11_Video.mp4', 'S12_Video.mp4']

In [77]:
MEDIA_TO_VIDEO = {
    # Your original main Media 1-12
    'Media 1': base_videos[0], 'Media 2': base_videos[1], 'Media 3': base_videos[2],
    'Media 4': base_videos[3], 'Media 5': base_videos[4], 'Media 6': base_videos[5],
    'Media 7': base_videos[6], 'Media 8': base_videos[7], 'Media 9': base_videos[8],
    'Media 10': base_videos[9], 'Media 11': base_videos[10], 'Media 12': base_videos[11],

    # Skipped/Extra from log (cycled based on Film N → base_videos[(N-1) % 12])
    'Media 2 (Film 17)': base_videos[(17-1) % 12],  # S5_Video.mp4
    'Media 3 (Film 22)': base_videos[(22-1) % 12],  # S10_Video.mp4
    'Media 4 (Film 23)': base_videos[(23-1) % 12],  # S11_Video.mp4
    'Media 5 (Film 25)': base_videos[(25-1) % 12],  # S1_Video.mp4
    'Media 6 (Film 27)': base_videos[(27-1) % 12],  # S3_Video.mp4
    'Media 8 (Film 13)': base_videos[(13-1) % 12],  # S1_Video.mp4
    'Media 9 (Film 14)': base_videos[(14-1) % 12],  # S2_Video.mp4
    'Media 12 (Film 19)': base_videos[(19-1) % 12],  # S7_Video.mp4
    'Media 13 (Film 28)': base_videos[(28-1) % 12],  # S4_Video.mp4
    'Media 14 (Film 26)': base_videos[(26-1) % 12],  # S2_Video.mp4
    'Media 15': base_videos[2],  # No Film; S3_Video.mp4 (sequential after 12)
    'Media 17 (Film 16)': base_videos[(16-1) % 12],  # S4_Video.mp4
    'Media 18 (Film 15)': base_videos[(15-1) % 12],  # S3_Video.mp4
    'Media 19 (Film 21)': base_videos[(21-1) % 12],  # S9_Video.mp4
    'Media 20 (Film 18)': base_videos[(18-1) % 12],  # S6_Video.mp4
    'Media 21': base_videos[8],  # No Film; S9_Video.mp4
    'Media 23 (Film 24)': base_videos[(24-1) % 12],  # S12_Video.mp4
    'Media 25': base_videos[0],  # No Film; S1_Video.mp4
    'Media 26 (Film 20)': base_videos[(20-1) % 12],  # S8_Video.mp4
}

print("MEDIA_TO_VIDEO dictionary updated with all skipped Media/Film mappings (using S1-S12 naming, cycled through 12 videos).")

MEDIA_TO_VIDEO dictionary updated with all skipped Media/Film mappings (using S1-S12 naming, cycled through 12 videos).


In [65]:
EYES_AUS = [
    'AD1', 'AD133', 'AD160', 'AD1L',
    'AU101', 'AU101L', 'AU145', 'AU145 L',
    'AU47', 'AU47L', 'AU5', 'AU5L'
    ]
CHIN_AUS = [
    'AD38', 'AU10', 'AU113', 'AU17', 'AU18', 'AU24'
    ]
EYES_PATTERN = '|'.join(EYES_AUS)
CHIN_PATTERN = '|'.join(CHIN_AUS)

#### ----------- FUNCTIONS -----------

In [78]:
def extract_clip(video_path, start_sec, end_sec, output_path, min_dur=0.5, max_dur=3.0):
    """Extracts segment from video and saves as output_path (mp4)."""
    dur = end_sec - start_sec
    if dur < min_dur:
        padding = (min_dur - dur)/2
        start_sec = max(0, start_sec - padding)
        end_sec = start_sec + min_dur
    elif dur > max_dur:
        start_sec += (dur - max_dur)/2
        end_sec = start_sec + max_dur

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if not fps or fps < 1:
        print(f"[WARNING] No FPS detected for: {video_path}")
        return False

    start_frame = int(start_sec * fps)
    end_frame = int(end_sec * fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    for _ in range(end_frame - start_frame):
        ret, frame = cap.read()
        if ret:
            out.write(frame)
        else:
            break
    cap.release()
    out.release()
    return True


In [79]:
def parse_s3_media_segments(s3_path, eyes_pat, chin_pat):
    raw = pd.read_excel(s3_path, sheet_name='FACS', header=None)
    media_dfs = {}
    current_media = None
    start_row = 0
    for idx, row in raw.iterrows():
        cell0 = str(row[0]).strip() if pd.notna(row[0]) else ''
        if cell0.startswith("Media"):
            if current_media is not None:
                header_row = start_row + 3
                nrows = idx - header_row
                media_df = pd.read_excel(
                    s3_path, sheet_name='FACS',
                    skiprows=header_row, nrows=nrows
                )
                media_dfs[current_media] = media_df
            current_media = cell0
            start_row = idx
    if current_media is not None:
        header_row = start_row + 3
        media_df = pd.read_excel(s3_path, sheet_name='FACS', skiprows=header_row)
        media_dfs[current_media] = media_df

    def pick_col(cols, keyword):
        for c in cols:
            if keyword in str(c).lower():
                return c
        return None

    def to_sec(t):
        if isinstance(t, str) and ":" in t:
            parts = [float(x) for x in t.split(":")]
            return parts[0]*3600 + parts[1]*60 + parts[2]
        try:
            return float(t)
        except Exception:
            return 0.0

    eyes_segments, chin_segments = {}, {}
    for media, df in media_dfs.items():
        code_col = pick_col(df.columns, 'code')
        begin_col = pick_col(df.columns, 'begin')
        end_col = pick_col(df.columns, 'end')
        if code_col and begin_col and end_col:
            eyes_df = df[df[code_col].astype(str).str.contains(eyes_pat, na=False)]
            chin_df = df[df[code_col].astype(str).str.contains(chin_pat, na=False)]
            eyes_begin = eyes_df[begin_col].apply(to_sec)
            eyes_end = eyes_df[end_col].apply(to_sec)
            eyes_segments[media] = list(zip(eyes_begin, eyes_end))
            chin_begin = chin_df[begin_col].apply(to_sec)
            chin_end = chin_df[end_col].apply(to_sec)
            chin_segments[media] = list(zip(chin_begin, chin_end))
    return eyes_segments, chin_segments

In [80]:
def parse_s2_film_segments(s2_path, eyes_pat, chin_pat):
    s2_df = pd.read_excel(s2_path, sheet_name='FACS')
    film_cols = [col for col in s2_df.columns if 'code' in str(col).lower()]
    eyes_segments, chin_segments = {}, {}
    for col in film_cols:
        film_name = next((f for f in s2_df.columns if f in str(col)), None)
        eyes_rows = s2_df[s2_df[col].astype(str).str.contains(eyes_pat, na=False)]
        chin_rows = s2_df[s2_df[col].astype(str).str.contains(chin_pat, na=False)]
        begin_col = pick_col(s2_df.columns, 'begin')
        end_col = pick_col(s2_df.columns, 'end')
        def to_sec(t):
            if isinstance(t, str) and ":" in t:
                parts = [float(x) for x in t.split(":")]
                return parts[0]*3600 + parts[1]*60 + parts[2]
            try:
                return float(t)
            except Exception:
                return 0.0
        if not (begin_col and end_col):
            continue
        eyes_begin = eyes_rows[begin_col].apply(to_sec)
        eyes_end = eyes_rows[end_col].apply(to_sec)
        chin_begin = chin_rows[begin_col].apply(to_sec)
        chin_end = chin_rows[end_col].apply(to_sec)
        eyes_segments[film_name] = list(zip(eyes_begin, eyes_end))
        chin_segments[film_name] = list(zip(chin_begin, chin_end))
    return eyes_segments, chin_segments

#### ----------- MAIN LOGIC -----------

In [81]:
# ----------- SEGMENT PARSING -----------

# --- Parse segments independently --
eyes_segments_s3, chin_segments_s3 = parse_s3_media_segments(S3_PATH, EYES_PATTERN, CHIN_PATTERN)
eyes_segments_s2, chin_segments_s2 = parse_s2_film_segments(S2_PATH, EYES_PATTERN, CHIN_PATTERN)
print("S3 Eyes segments keys:", list(eyes_segments_s3.keys()))
print("S3 Chin segments keys:", list(chin_segments_s3.keys()))


S3 Eyes segments keys: ['Media 1', 'Media 2 (Film 17)', 'Media 3 (Film 22)', 'Media 4 (Film 23)', 'Media 5 (Film 25)', 'Media 6 (Film 27)', 'Media 7', 'Media 8 (Film 13)', 'Media 9 (Film 14)', 'Media 12 (Film 19)', 'Media 13 (Film 28)', 'Media 14 (Film 26)', 'Media 15', 'Media 17 (Film 16)', 'Media 18 (Film 15)', 'Media 19 (Film 21)', 'Media 20 (Film 18)', 'Media 21', 'Media 23 (Film 24)', 'Media 25', 'Media 26 (Film 20)']
S3 Chin segments keys: ['Media 1', 'Media 2 (Film 17)', 'Media 3 (Film 22)', 'Media 4 (Film 23)', 'Media 5 (Film 25)', 'Media 6 (Film 27)', 'Media 7', 'Media 8 (Film 13)', 'Media 9 (Film 14)', 'Media 12 (Film 19)', 'Media 13 (Film 28)', 'Media 14 (Film 26)', 'Media 15', 'Media 17 (Film 16)', 'Media 18 (Film 15)', 'Media 19 (Film 21)', 'Media 20 (Film 18)', 'Media 21', 'Media 23 (Film 24)', 'Media 25', 'Media 26 (Film 20)']


In [82]:
# --- Merge results for extraction ---
eyes_segments = eyes_segments_s3.copy()
chin_segments = chin_segments_s3.copy()
for film in eyes_segments_s2:
    media_key = film.replace('FILM ', 'Media ')
    if media_key not in eyes_segments:
        eyes_segments[media_key] = eyes_segments_s2[film]
        chin_segments[media_key] = chin_segments_s2[film]
    else:
        eyes_segments[media_key].extend(eyes_segments_s2[film])
        chin_segments[media_key].extend(chin_segments_s2[film])

clip_metadata = []

In [83]:
# --- Main Execution Loop ---
for media in eyes_segments:
    video_file = MEDIA_TO_VIDEO.get(media)
    if not video_file:
        print(f"[SKIP] No video mapping for: {media}")
        continue
    video_path = os.path.join(VIDEOS_DIR, video_file)
    if not os.path.isfile(video_path):
        print(f"[SKIP] Video not found: {video_path}")
        continue
    # --- Eyes Positive ---
    for i, (start, end) in enumerate(eyes_segments.get(media, [])):
        out_path = os.path.join(CLIPS_DIR, 'eyes_positive', f"{media}_clip{i}.mp4")
        ok = extract_clip(video_path, start, end, out_path)
        if ok:
            clip_metadata.append({'clip': out_path, 'eyes_present': 1, 'chin_present': 0, 'nostrils_present': None})
    # --- Chin Positive ---
    for i, (start, end) in enumerate(chin_segments.get(media, [])):
        out_path = os.path.join(CLIPS_DIR, 'chin_positive', f"{media}_clip{i}.mp4")
        ok = extract_clip(video_path, start, end, out_path)
        if ok:
            clip_metadata.append({'clip': out_path, 'eyes_present': 0, 'chin_present': 1, 'nostrils_present': None})
    # --- Negative Clips ---
    cap = cv2.VideoCapture(video_path)
    total_dur = cap.get(cv2.CAP_PROP_FRAME_COUNT) / (cap.get(cv2.CAP_PROP_FPS) or 1)
    cap.release()
    all_pos = sorted(set(eyes_segments.get(media, []) + chin_segments.get(media, [])))
    gaps = [(0, all_pos[0][0])] if all_pos else [(0, total_dur)]
    for j in range(len(all_pos)-1):
        gaps.append((all_pos[j][1], all_pos[j+1][0]))
    if all_pos:
        gaps.append((all_pos[-1][1], total_dur))
    gaps = [g for g in gaps if g[1] - g[0] >= 0.5]
    num_neg = min(145, len(gaps))
    for i in range(num_neg):
        gap = random.choice(gaps)
        max_len = min(3.0, gap[1] - gap[0])
        if max_len < 0.5:
            continue
        start = random.uniform(gap[0], gap[1] - max_len)
        end = start + random.uniform(0.5, max_len)
        out_path = os.path.join(CLIPS_DIR, 'negative', f"{media}_neg_clip{i}.mp4")
        ok = extract_clip(video_path, start, end, out_path)
        if ok:
            clip_metadata.append({'clip': out_path, 'eyes_present': 0, 'chin_present': 0, 'nostrils_present': None})

In [84]:
# --- SPLIT & SAVE ---
CSV_DIR = os.path.join(project_root, 'outputs', 'csv')
clip_df = pd.DataFrame(clip_metadata)
if not clip_df.empty:
    labels = clip_df['eyes_present'] + 2 * clip_df['chin_present']
    train, temp = train_test_split(clip_df, test_size=0.3, stratify=labels, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, stratify=temp['eyes_present'] + 2 * temp['chin_present'], random_state=42)

    train.to_json(os.path.join(CSV_DIR, 'train_metadata.json'), orient='records')
    val.to_json(os.path.join(CSV_DIR, 'val_metadata.json'), orient='records')
    test.to_json(os.path.join(CSV_DIR, 'test_metadata.json'), orient='records')
    train.to_csv(os.path.join(CSV_DIR, 'train_metadata.csv'), index=False)
    val.to_csv(os.path.join(CSV_DIR, 'val_metadata.csv'), index=False)
    test.to_csv(os.path.join(CSV_DIR, 'test_metadata.csv'), index=False)

    print(f"Task 1 complete: {len(clip_df)} total clips ({len(train)} train, {len(val)} val, {len(test)} test).")
else:
    print("No clips extracted—check segment extraction and video mapping.")

Task 1 complete: 1211 total clips (847 train, 182 val, 182 test).
